In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
from time import sleep

In [2]:
search_item = "banana"
url = "https://www.tesco.com/groceries/en-GB/search?query="

In [3]:
driver = webdriver.Chrome('./chromedriver')
driver.get(url+search_item)

In [4]:
# ele = driver.find_element_by_xpath("//*[@id='product-list']/div[2]/div[3]/div[1]/div/div[1]/div[1]/div[2]/a")
# ele.click()

In [5]:
soup = bs(driver.page_source)

In [6]:
column_names = c
data = pd.DataFrame(columns = column_names)

In [8]:
def search_tesco(search_item, driver=None):
    url = "https://www.tesco.com/groceries/en-GB/search?query="
    search_url = url+search_item
    
    driver = webdriver.Chrome('./chromedriver') if not driver else driver
    driver.get(search_url)
#     cookie_clicked = False
#     while not cookie_clicked:
#         try:
#             driver.find_element_by_xpath("//*[@id=\"onetrust-accept-btn-handler\"]").click()
#             cookie_clicked = True
#         except:
#             sleep(0.5)
            
    column_names = ["product_name", "total_price", "unit_price", "img", "link"]
    data = pd.DataFrame(columns = column_names)
    
    sleep_count = 0
    sleep_len = 0.5
    max_sleep_count = 5 // sleep_len
    sleep(0.5)
    
    while True:
        soup = bs(driver.page_source)
    
        try:
            section = soup.find("ul", {"class": "product-list grid"})
            products = section.find_all("li", {"class": "product-list--list-item"})
            if (len(products)!=0):
                break
            sleep_count += 1
            sleep(0.5)
            if (sleep_count >= max_sleep_count):
                print("Can't find product!")
                return data
        except IndexError:
            return data
        except:
            print("Unknown error")
        
    domain = "https://www.tesco.com"
   
    for product in products[:24]:
        try:
            name = product.find("a", {"class": "ui__StyledLink-sc-18aswmp-0 bfYkKW"}).text
            link = product.find("a", {"class": "ui__StyledLink-sc-18aswmp-0 bfYkKW"})["href"]
            product_link = domain+link
            total_price_element = product.find("div", {"class": "price-per-sellable-unit"})
            currency = total_price_element.find("span", {"class":"currency"}).text
            price = total_price_element.find("span", {"class": "value"}).text
            total_price_info = currency+price
            unit_price_element = product.find("div", {"class": "price-per-quantity-weight"})
            unit_currency = unit_price_element.find("span", {"class":"currency"}).text
            unit_price = unit_price_element.find("span", {"class": "value"}).text
            unit_weight = unit_price_element.find("span", {"class": "weight"}).text
            unit_price_info = unit_currency+unit_price+unit_weight
            image = product.find("img", {"class": "product-image"})['src']
            data_row = pd.Series([name, total_price_info, unit_price_info, image, product_link], index = column_names)
            data = data.append(data_row, ignore_index = True)
        except: 
            continue
    
    return data

In [9]:
def search_sainsburys(search_item, driver = None):
    url = "https://www.sainsburys.co.uk/gol-ui/SearchDisplayView?filters[keyword]="
    search_url = url+search_item
    
    driver = webdriver.Chrome('./chromedriver') if not driver else driver

    sleep_count = 0
    sleep_len = 0.5
    max_sleep_count = 2 // sleep_len
        
    driver.get(search_url)
    cookie_clicked = False
    while not cookie_clicked:
        try:
            driver.find_element_by_xpath("//*[@id=\"onetrust-accept-btn-handler\"]").click()
            cookie_clicked = True
        except:
            sleep_count += 1
            sleep(sleep_len)
            if (sleep_count >= max_sleep_count):
                break
            
    column_names = ["product_name", "total_price", "unit_price", "img", "link"]
    data = pd.DataFrame(columns = column_names)

    sleep_len = 0.5
    
    soup = bs(driver.page_source)
    header_ele = soup.find("h1", {"class": "si__title"})
    header = header_ele.text if header_ele else "Fetching results"
    while "Fetching results" in header:
        print(header)
        sleep(sleep_len)
        soup = bs(driver.page_source)
        header_ele = soup.find("h1", {"class": "si__title"})
        header = header_ele.text if header_ele else "Fetching results"
        
    print(header)
    
    if "0 results" in header:
        return data

    section_ele = soup.find_all("section", {"class": "ln-o-section ln-o-section"})
    while not section_ele:
        sleep(sleep_len)
        section_ele = soup.find_all("section", {"class": "ln-o-section ln-o-section"})
    section = section_ele[0]
    products = section.find_all("li", {"class": "pt-grid-item ln-o-grid__item ln-u-6/12@xs ln-u-3/12@md ln-u-2/12@xl"})
        
    for product in products[:60]:
        text_bar = product.find("h2", {"class": "pt__info__description"})
        img = product.find("img", {"class": "pt-image pt-image__product"})["src"]
        costs_bar = product.find("div", {"class": "pt__cost"})
        total_price = costs_bar.find("div").text
        unit_price = costs_bar.find("span").text
        name = text_bar.text
        link = text_bar.find("a", href=True)["href"]
        data_row = pd.Series([name, total_price, unit_price, img, link], index = column_names)
        data = data.append(data_row, ignore_index = True)
    
    return data

In [10]:
supported_supermarkets = ["Sainsburys", "Tesco"]
scraping_funcs = [search_sainsburys, search_tesco]
# supported_supermarkets = ["Tesco"]
# scraping_funcs = [search_tesco]

supermarket_dict = {supermarkets:scraping_funcs[idx] for idx, supermarkets in enumerate(supported_supermarkets)}

In [19]:
driver = webdriver.Chrome('./chromedriver')

shopping_cart = ["eggs", "banana", "cumin powder", "spatula", "frog", "milk", "bread", "chocolate"]
scraped_data = {i:{} for i in shopping_cart}

for item in shopping_cart:
    for supermarket in supermarket_dict:
        scrape_func = supermarket_dict[supermarket]
        scraped_data[item][supermarket] = scrape_func(item, driver)
    
scraped_data

Fetching results
Fetching results
Fetching results for ‘eggs’…
Fetching results for ‘eggs’…
Fetching results for ‘eggs’…
Fetching results for ‘eggs’…
Fetching results for ‘eggs’…
385 results for ‘eggs’
132 results for ‘banana’
0 results for ‘cumin powder’
6 results for ‘spatula’
2 results for ‘frog’
Fetching results for ‘milk’…
Fetching results for ‘milk’…
863 results for ‘milk’
324 results for ‘bread’
Fetching results for ‘chocolate’…
1331 results for ‘chocolate’


{'eggs': {'Sainsburys':                                          product_name total_price  \
  0            J.James Free Range Eggs Mixed Weight x10       £1.10   
  1   Sainsbury's Woodland Free Range Golden Yolked ...       £1.90   
  2                    Two Chicks Liquid Egg White 500g       £2.50   
  3      Sainsbury's Woodland Free Range Large Eggs x12       £2.05   
  4       Sainsbury's Woodland Free Range Large Eggs x6       £1.10   
  5     Sainsbury's Woodland Free Range Medium Eggs x12       £1.70   
  6      Sainsbury's Woodland Free Range Medium Eggs x6         90p   
  7   Sainsbury's Woodland Free Range Large Eggs, SO...       £2.15   
  8                  Happy Egg Large Free Range Eggs x6       £1.75   
  9   Sainsbury's Woodland Free Range Medium Eggs, S...       £1.80   
  10                Happy Egg Large Free Range Eggs x10       £2.75   
  11  Sainsbury's Woodland Free Range Very Large Egg...       £1.60   
  12  Clarence Court Burford Browns Free Range Eggs ...

In [12]:
scraped_data['eggs']['Sainsburys']

,product_name,total_price,unit_price,img,link
0,J.James Free Range Eggs Mixed Weight x10,£1.10,11p / ea,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...
1,Sainsbury's Woodland Free Range Golden Yolked ...,£1.90,32p / ea,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...
2,Two Chicks Liquid Egg White 500g,£2.50,50p / 100g,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...
3,Sainsbury's Woodland Free Range Large Eggs x12,£2.05,17p / ea,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...
4,Sainsbury's Woodland Free Range Large Eggs x6,£1.10,18p / ea,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...
5,Sainsbury's Woodland Free Range Medium Eggs x12,£1.70,14p / ea,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...
6,Sainsbury's Woodland Free Range Medium Eggs x6,90p,15p / ea,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...
7,"Sainsbury's Woodland Free Range Large Eggs, SO...",£2.15,36p / ea,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...
8,Happy Egg Large Free Range Eggs x6,£1.75,29p / ea,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...
9,"Sainsbury's Woodland Free Range Medium Eggs, S...",£1.80,30p / ea,https://assets.sainsburys-groceries.co.uk/gol/...,https://www.sainsburys.co.uk/shop/gb/groceries...


In [20]:
all_df = pd.DataFrame(columns=["product_name", "total_price", "unit_price", "img", "link", "supermarket"])
for item, inner_dict in scraped_data.items():
    product_name = item
    print(product_name)
    for supermarket, dataframe in inner_dict.items():
        print(supermarket)
        dataframe.info()
        dataframe["supermarket"] = supermarket
        all_df = all_df.append(dataframe)

eggs
Sainsburys
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  60 non-null     object
 1   total_price   60 non-null     object
 2   unit_price    60 non-null     object
 3   img           60 non-null     object
 4   link          60 non-null     object
dtypes: object(5)
memory usage: 2.5+ KB
Tesco
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  23 non-null     object
 1   total_price   23 non-null     object
 2   unit_price    23 non-null     object
 3   img           23 non-null     object
 4   link          23 non-null     object
dtypes: object(5)
memory usage: 1.0+ KB
banana
Sainsburys
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (tota

In [21]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434 entries, 0 to 23
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  434 non-null    object
 1   total_price   434 non-null    object
 2   unit_price    434 non-null    object
 3   img           434 non-null    object
 4   link          434 non-null    object
 5   supermarket   434 non-null    object
dtypes: object(6)
memory usage: 23.7+ KB


In [33]:
all_df.loc[all_df.total_price == '73p / kg', 'total_price'] = '73p'

In [37]:
def clean_price(x):
    x = x.replace('£', '').replace('p', '').replace('.', '')
    return(x)
all_df['total_price'] = all_df['total_price'].apply(clean_price).astype(float)

In [38]:
all_df.to_csv('out.csv', index=False)